# Introduction 

This notebook focuses on applying the [Variational Quantum Eigensolver](https://learning.quantum.ibm.com/tutorial/variational-quantum-eigensolver) (VQE) algorithm to solve the [Traveling Salesman Problem (TSP)](https://en.wikipedia.org/wiki/Travelling_salesman_problem), an [NP-Complete](https://en.wikipedia.org/wiki/NP-completeness) problem which has been in computer science literature since a long time. A very straightforward definition of the problem could be the following: 

<p style="text-align: center; bold"> <b> Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city exactly once and returns to the origin city? </b> </p>

Think about such problem, for a user which has thousands of cities and routes to analyze: it becomes immediately clear how easily the computation time increases. In order to solve TSP using quantum computers, we first need to rethink the problem definition according to quantum mechanics. Since TSP is a [combinatorial optimization](https://en.wikipedia.org/wiki/Combinatorial_optimization) problem, we can use the Quadratic unconstrained binary optimization (QUBO) formulation, which is specifically designed to exploit quantum computation to solve classical combinatorial optimization problems: oversimplifying, QUBO formulation rewrites the problem using a different cost function, which includes the constraints in form of quadratic penalties, since QUBO formulation is, as the name suggests, unconstrained. Once a generic combinatorial problem is expressed in a QUBO formulation, we can transform the problem once more, by leveraging the [universality class](https://en.wikipedia.org/wiki/Universality_class) of the Ising Model. Using Ising Model to solve combinatorial optimization problems is convenient in this context since it provides a straightforward way to represent the problem according to quantum mechanics laws and thus it let us use the quantum computer power to solve it.



### Install dependencies and check qiskit version 

First, let us install all the needed dependencies and check that we are using Qiskit 1.0.2 version 

In [ ]:
# Install packages in the current Jupyter kernel

import sys
!{sys.executable} -m pip install qiskit==1.0.2
!{sys.executable} -m pip install qiskit-aer==0.14.2
!{sys.executable} -m pip install qiskit-algorithms==0.3.1
!{sys.executable} -m pip install qiskit-ibm-provider==0.11.0
!{sys.executable} -m pip install qiskit-ibm-runtime==0.23.0
!{sys.executable} -m pip install qiskit-machine-learning==0.8.2
!{sys.executable} -m pip install qiskit-optimization==0.6.1

In [ ]:
# Basic imports (all others will be imported when needed for teaching purposes)

import numpy as np
import matplotlib.pyplot as plt

import qiskit.version
print("You are using Qiskit "+qiskit.version.get_version_info())


### Utility functions and classical problem definition
Let us define some utility functions and the problem in a classical way by using networkx as graph utility library and qiskit_optimization.applications Tsp object representation. This gives us a convenient python object to represent the TSP and also to natively interpret the solutions provided by the quantum computer as graph routes

In [ ]:
import networkx as nx
from qiskit_optimization.applications import Tsp


# Utility function to draw an input graph
def draw_graph(G, colors, pos):
    default_axes = plt.axes(frameon=True)
    nx.draw_networkx(G, node_color=colors, node_size=600, alpha=0.8, ax=default_axes, pos=pos)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels)
    plt.title("Cities and route lengths", fontsize=12)


# Utility function to highlight a graph path on a given input graph
def draw_tsp_solution(G, order, colors, pos, title):
    G2 = nx.DiGraph()
    G2.add_nodes_from(G)
    n = len(order)
    for i in range(n):
        j = (i + 1) % n
        G2.add_edge(order[i], order[j], weight=G[order[i]][order[j]]["weight"])
    default_axes = plt.axes(frameon=True)
    nx.draw_networkx(G2, node_color=colors, edge_color="b", node_size=600, alpha=0.8, ax=default_axes, pos=pos)
    edge_labels = nx.get_edge_attributes(G2, "weight")
    nx.draw_networkx_edge_labels(G2, pos, font_color="b", edge_labels=edge_labels)
    plt.title("Best route found - "+title, fontsize=12)
 


In [ ]:
# Problem definition: we define a new random graph with n nodes
n = 5
tsp_problem = Tsp.create_random_instance(n)
G = tsp_problem.graph

# Since in this example we keep the graph small, is a good thing to draw it, so we can verify the solution also graphically
adj_matrix = nx.to_numpy_array(G)
print("distance\n", adj_matrix)


# Define a list of base colors (for initial nodes)
base_colors = ["red", "blue", "green", "orange", "purple", "cyan", "magenta", "yellow"]

# If n exceeds the length of the base_colors list, generate random colors
if n > len(base_colors):
    # Generate random colors for remaining nodes
    additional_colors = [f"#{random.randint(0, 0xFFFFFF):06x}" for _ in range(n - len(base_colors))]
    colors = base_colors + additional_colors
else:
    colors = base_colors[:n]

pos = nx.spring_layout(G)

fig, ax = plt.subplots(1, figsize=(7, 5))
draw_graph(tsp_problem.graph, colors, pos)


### Conversion to QUBO and Ising Model

This is probably the most important section of the example. Here the TSP problem representation is natively converted into a QUBO formulation by usign the qiskit_optimization library. Note that this is done transparently and without the need to implement custom code. Another crucial step performed automatically by the Qiskit framework is the conversion to the Ising Model. This conversion, as stated in the intro, is very useful to get a problem definition that is expressed in terms of quantum related concepts. Without it, we should rethink a whole new problem definition and information encoding strategies. By leveraging the powerfulness of the Ising model, we can not only map the TSP into a quantum circuit automatically, but we can do that also for many other combinatorial optimization problems!  

In [ ]:
from qiskit_optimization.converters import QuadraticProgramToQubo


qp = tsp_problem.to_quadratic_program()
qp2qubo = QuadraticProgramToQubo()
qubo = qp2qubo.convert(qp)

# Converts QUBO into an Ising model and extracts the Hamiltonian Observable 
ising_hamiltonian, offset = qubo.to_ising()


### Solving te problem with a classical solver for later comparison

In [ ]:
from qiskit_algorithms import NumPyMinimumEigensolver

# Here we solve the problem with a classical solver to have an exact solution to compare with the VQE results 

ee = NumPyMinimumEigensolver()
result = ee.compute_minimum_eigenvalue(ising_hamiltonian)

print("energy:", result.eigenvalue.real)
print("tsp objective:", result.eigenvalue.real + offset)
x = tsp_problem.sample_most_likely(result.eigenstate)
print("feasible:", qubo.is_feasible(x))
z= tsp_problem.interpret(x)
print("solution:", z)
print("solution objective:", tsp_problem.tsp_value(z, adj_matrix))


### Ansatz definition

If QUBO/Ising mapping was the most important part of the example, the Ansatz definition follows it right after.

Ansatz is a german word to say tentative, or approach, and it is used in this context because it actually represents all the possible solutions of the TSP problem, in a parametric quantum circuit. For the TSP there are many articles in literature that define possible ansatz, the one that follows is one of the results of a doctoral thesis in quantum computation.

##### Problem-Specific Parametric Quantum Circuit (PQC)

In this [work](https://arxiv.org/pdf/2006.05643) is introduced the general concept of the Problem-Specific Parametric Quantum Circuit (PQC).

After mapping each binary variable $x_i$ to the qubit $q_i$, our focus shifts to the constraints of an optimization problem. These constraints eﬀectively limit the set of feasible solutions for the optimization problem: by leveraging these constraints, we dynamically construct a problem-specific PQC that reflects the optimization problem’s constraints.

This strategic approach allows us to restrict the unitary transformation provided by the problem-specific PQC while considering the defined constraints. Consequently, we can eﬀectively reduce the set of basis states in the output state vector of the problem-specific PQC, significantly shrinking the search space.



In [ ]:
from qiskit.circuit import QuantumCircuit, ParameterVector

# Initialization of the base Ansatz circuit
def base_circuit(QC, n, theta):
    theta1 = ParameterVector('theta2', 1)
    QC.x(0)
    QC.ry(theta[1], 1)
    QC.cz(0, 1)
    QC.ry(-theta[1], 1)
    QC.cx(1, 0)
    QC.cx(1, n)
    QC.cx(0, n + 1) 
    
# y axis rotation and controlled gates to encode all possible solutions of the problem in the Ansatz circuit
def W_circuit(QC, n, q1n, theta):
    QC.x(q1n)
    for j in range(q1n + 1, q1n + n, 1):
        QC.ry(theta[j - 1], j)
        QC.cz(j - 1, j)
        QC.ry(-theta[j - 1], j)
    for j in range(q1n + 1, q1n + n, 1):
        QC.cx(j, j - 1)

num_qubits = n ** 2
qc = QuantumCircuit(num_qubits)
phi = ParameterVector('phi', num_qubits)

# The following lines define a custom Ansatz for the TSP using the above base_circuit and W_circuit 
base_circuit(qc, n, phi)

for i in range(3, n + 1, 1):
    W_circuit(qc, i, n * (i - 1), phi)
for k in range(3, n + 1, 1):
    for v in range(1, k, 1):
        for p in range(1, k, 1):
            qc.cswap(n * (k - 1) + v - 1, n * (p - 1) + n - (n - k) - 1, n * (p - 1) + v - 1)
qc.measure_all()

ansatz = qc


### Now that we have all the ingredients for our recipe, let us cook it using quantum computation

First, let us define the Qiskit backend. For the sake of clarity, we will use a noiseless simulator this time. This way, we can verify that the results are the ones expected and thus prove that the formulation above is correct. 

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Select the platform: IBM Quantum (quantum.ibm.com) or IBM Cloud (quantum.cloud.ibm.com)
channel = 'ibm_cloud' #ibm_cloud, #ibn_quantum

if channel == 'ibm_quantum':
  service = QiskitRuntimeService(
    channel = channel,
    # IBM Quantum token
    token = '',
  )

elif channel == 'ibm_cloud':
  service = QiskitRuntimeService(
    channel = channel,
    # IBM Cloud API key
    token = '',
    # IBM Cloud CRN
    instance = ''
  )

In [ ]:
# List all backend names
backends = service.backends()
print([backend.name for backend in backends])

In [ ]:
from qiskit_aer import AerSimulator

# Run type configuration
run_target = 'simulator' #simulator, #least_busy, #any hw printed above

if run_target == 'simulator':
    backend = AerSimulator()

elif run_target == 'least_busy':
    backend = service.least_busy(operational=True, simulator=False)

else:
    backend = service.backend('ibm_sherbrooke')

print("Selected channel: "+channel)
print("Selected backend: "+backend.name)

if ((channel == 'ibm_quantum') and (run_target != 'simulator')):
    print ("-- WARNING: This run may consume several minutes of your Open plan")
elif ((channel == 'ibm_cloud') and (run_target != 'simulator')):
    print ("-- WARNING: This run may consume several credits of your Paygo plan: ~1.6$/second")
elif (run_target == 'simulator'):
    print ("-- This run will be a quick simulation using Qiskit Aer")


Now we transpile the ansatz circuit in order to make it executable in an optimized way on our backend. Note how easy is to perform this operation with the new version of Qiskit: the only thing needed is a backend and a circuit definition! With these two, we can leverage the qiskit.transpiler library to optimize our circuit automatically.

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit import transpile

if(backend == 'simulator'):
    pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
    isa_ansatz = pm.run(ansatz)
else:
    isa_ansatz = transpile(ansatz, backend=backend)


# Ansatz drawing
isa_ansatz.decompose().draw("mpl", scale=0.8, plot_barriers=False)


####  The resolution of QUBO

To solve the QUBO problem, we use a hybrid classical/quantum approach: the ansatz and the Hamiltonian observable are evaluated with the quantum computer, while the parameters space used to evaluate the ansatz (which we will use later to determine the lowest energy of the system that translates directly into the shortest path of our salesman) is explored through a classical algorithm. This allows to leverage the best capabilities of each technology: classical computer to perform repetitive small tasks and quantum computer for hard one-time calculations.   

In [ ]:
# Define cost function
cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
   }
    
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance
        cost_history_dict: Dictionary for storing intermediate results

    Returns:
        float: Energy estimate
    """
   # Here you can see the code that evaluates the ansatz with a specific set of parameters and then measures the observable hamiltonian.
    pub = (ansatz, [hamiltonian], [params])
    partial_result = estimator.run(pubs=[pub]).result()
    energy = partial_result[0].data.evs[0]

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)
    print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]")

    return energy

To implement this hybrid/quantum approach we use scipy library, which exposes convenient features to map combinatorial optimization problems. 

In [ ]:
from qiskit_algorithms.utils import validate_initial_point
from qiskit_ibm_runtime import Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from scipy.optimize import minimize


num_params = ansatz.num_parameters
x0 = validate_initial_point(None, isa_ansatz)
hamiltonian_isa = ising_hamiltonian.apply_layout(isa_ansatz.layout)



with Session(backend=backend) as session:
    
    estimator = Estimator(session=session)
    estimator.options.default_shots = 3000
    
    res = minimize(
        cost_func,
        x0,
        args=(isa_ansatz, hamiltonian_isa, estimator),
        method="COBYLA",
        tol=1e-7,
        options={'maxiter': 200}
    )
    session.close()


Finally, we extract the results by evaluating the ansatz with the parameters that minimize the Hamiltonian of the system with a Sampler. This will give us the solution to be interpreted natively by the TSP object and in the end we compare the results with the classical ones. 

In [ ]:
from qiskit_ibm_runtime import SamplerV2 as Sampler

print("Results of the optimization -" + str(res))

with Session(backend=backend) as session:
    sampler = Sampler(session=session)

    pub1 = (isa_ansatz, res.x)    
    final_state = sampler.run(pubs=[pub1]).result()
    quantum_solution = tsp_problem.interpret(tsp_problem.sample_most_likely(final_state._pub_results[0].data.meas.get_counts()))
    print("Solution found with VQE: " + str(quantum_solution) + " with cost: " + str(tsp_problem.tsp_value(quantum_solution, adj_matrix)))
    print("Solution found with classical solver: " + str(z) + " with cost: " + str(tsp_problem.tsp_value(z, adj_matrix)))

    # Cities and route lengths
    fig1 = plt.figure(figsize=(5,4))
    draw_graph(tsp_problem.graph, colors, pos)

    # TSP classical solution
    fig2 = plt.figure(figsize=(5,4))
    draw_tsp_solution(tsp_problem.graph, z, colors, pos, "Classical")

    # TSP quantum solution
    fig3 = plt.figure(figsize=(5,4))
    draw_tsp_solution(tsp_problem.graph, quantum_solution, colors, pos, "Quantum")
    

In [ ]:
all(cost_history_dict["prev_vector"] == res.x)

fig, ax = plt.subplots()
ax.plot(range(cost_history_dict["iters"]), cost_history_dict["cost_history"])
ax.set_xlabel("Iterations")
ax.set_ylabel("Cost")
plt.draw()

p = tsp_problem.tsp_value(quantum_solution, adj_matrix) # total route weight
print("solution objective:", p)